In [127]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from sklearn.svm import OneClassSVM
import numpy as np
from scipy import sparse

In [128]:
df_train = pd.read_csv(r'C:\Users\camil\OneDrive\Área de Trabalho\mestrado\treino\r2_bo_train_statements.csv', sep=';')
df_train = df_train.drop(df_train[df_train.Polarity =='other'].index)
df_train.Polarity.replace(('for', 'against'), (1,0), inplace=True)
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_train

,Text,Target,Polarity,Factual,User_ID,Tweet_ID,Tweet_Seq,POS,Mentions,Nouns
0,Bolsonaro já tá avisando que vai acabar com a ...,r2_bo,0,no,r2_bo_299,_1250880546441515010,5475,"[('bolsonaro', 'N'), ('já', 'ADV'), ('tá', 'V'...",[],"{'bolsonaro': 1, 'economia': 1, 'quarentena': ..."
1,"HIPOCRISIA É MATO, HOMOFOBIA MATA, IGNORÂNCIA ...",r2_bo,0,no,r2_bo_99,_1192689402209812480,4167,"[('hipocrisia', 'N'), ('é', 'V'), ('mato', 'N'...",['drikbarbosa'],"{'drikbarbosa': 1, 'bolsonaro': 1, 'hipocrisia..."
2,"Essa traíra pegou carona, está com inveja de J...",r2_bo,1,no,r2_bo_734,_1238107652556836864,1499,"[('essa', 'PROADJ'), ('traíra', 'N'), ('pegou'...",['joicehasselmann'],"{'traíra': 1, 'inveja': 1, 'figura': 1, 'caron..."
3,"Não me deixem só!”: Fora de controle, Bolsonar...",r2_bo,0,no,r2_bo_75,_1233897043426529280,2720,"[('“', 'N'), ('não', 'ADV'), ('me', 'PROPESS')...",[],"{'controle': 1, 'bolsonaro': 1, 'collor': 1, '..."
4,ADEMAR EU FIZ UMAS POSTAGENS A RESPEITO HOJE D...,r2_bo,1,no,r2_bo_737,_1159733091583369216,1364,"[('>', 'N'), ('>', 'PU'), ('ademar', 'NPROP'),...",['jairbolsonaro'],"{'life': 1, 'apoio': 1, 'sucesso': 1, 'guedes'..."
...,...,...,...,...,...,...,...,...,...,...
7055,Já estamos em 2019 e sob nova direção. bolsona...,r2_bo,1,no,r2_bo_722,_1079924121528659968,187,"[('já', 'ADV'), ('estamos', 'V'), ('em', 'PREP...",['jairbolsonaro'],"{'deus': 1, 'jairbolsonaro': 1, 'direção': 1, ..."
7056,Isso mesmo irmã o demônio esta cada vez mais f...,r2_bo,1,no,r2_bo_569,_1163621156022378497,1920,"[('isso', 'PROSUB'), ('mesmo', 'PDEN'), ('irmã...",[],"{'vez': 1, 'demônio': 1, 'irmã': 1, 'meios': 1}"
7057,"Não faz sentido não, mas por favor deixe bem c...",r2_bo,1,no,r2_bo_723,_1197920726377996289,2410,"[('sem', 'PREP'), ('contar', 'V'), ('que', 'KS...",['jairbolsonaro'],"{'pessoas': 1, 'país': 1, 'culpa': 1, 'ignorân..."
7058,PQP!!!Até quando vamos ter que aturar essa gen...,r2_bo,1,no,r2_bo_702,_1114994812502073344,470,"[('pqp', 'IN'), ('!', 'PU'), ('!', 'PU'), ('!'...",[],"{'militância': 1, 'governo': 1, 'bolsonaro': 1..."


In [129]:
df_test = pd.read_csv(r'C:\Users\camil\OneDrive\Área de Trabalho\mestrado\Teste\r2_bo_test_statements.csv', sep=';')
df_test = df_test.drop(df_test[df_test.Polarity =='other'].index)
df_test.Polarity.replace(('for', 'against'), (1,0), inplace=True)
df_test = df_test.sample(frac=1).reset_index(drop=True)
df_test

,Text,Target,Polarity,Factual,User_ID,Tweet_ID,Tweet_Seq,POS,Mentions,Nouns
0,A estupidez canhota não tem limites. A não ser...,r2_bo,1,no,r2_bo_724,_1092731615892127744,434,"[('a', 'ART'), ('estupidez', 'N'), ('canhota',...",[],"{'estupidez': 1, 'limites': 1, 'direita': 1, '..."
1,Bolsonaro é um merda. Esse eh o Tweet,r2_bo,0,no,r2_bo_91,_1250881841067941892,4733,"[('bolsonaro', 'N'), ('é', 'V'), ('um', 'ART')...",[],"{'merda': 1, 'bolsonaro': 1, 'tweet': 1}"
2,Acho que o futuro presidente poderia pensar na...,r2_bo,1,no,r2_bo_103,_1066757601575227393,28,"[('acho', 'V'), ('que', 'KS'), ('o', 'ART'), (...",['jairbolsonaro'],"{'povo': 2, 'plataforma': 1, 'intuito': 1, 'go..."
3,Esse @ nunca foi apoiador de Jair Messias Bols...,r2_bo,1,no,r2_bo_734,_1264189677168472064,2005,"[('esse', 'PROADJ'), ('@', 'NUM'), ('datenaofi...",['datenaoficial'],"{'datenaoficial': 1, 'picareta': 1, 'máscara':..."
4,Nosso Guerreiro é iluminado. Deus cuide de nos...,r2_bo,1,no,r2_bo_733,_1253090110159101953,427,"[('nosso', 'PROADJ'), ('guerreiro', 'N'), ('é'...",[],"{'guerreiro': 1, 'deus': 1, 'presidente': 1}"
...,...,...,...,...,...,...,...,...,...,...
2349,"CARLA, ESPERAMOS QUE O bolsonaro NÃO DIVIDA O ...",r2_bo,1,no,r2_bo_737,_1220432211276595201,2695,"[('>', 'N'), ('>', 'N'), ('carla', 'N'), (',',...","['sf_moro', 'sf_moro', 'pauloguedesbr']","{'sf_moro': 2, 'escolha': 1, 'sucesso': 1, 'pa..."
2350,Canalhas mil vezes canalhas !! Realmente o neg...,r2_bo,1,no,r2_bo_735,_1260213220440190979,1693,"[('canalhas', 'N'), ('mil', 'NUM'), ('vezes', ...",[],"{'bandidos': 2, 'vcs': 2, 'canalhas': 1, 'bols..."
2351,Eu fico pensando como as meninas que moram no ...,r2_bo,0,no,r2_bo_693,_1254160331754086400,3839,"[('eu', 'PROPESS'), ('fico', 'V'), ('pensando'...",[],"{'condomínio': 1, 'bolsonaro': 1, 'meninas': 1}"
2352,"bolsonaroístas"" podem ser chamados aqueles que...",r2_bo,0,no,r2_bo_559,_724328704663494656,1592,"[('``', 'N'), ('bolsonaroístas', 'N'), (""''"", ...",[],"{'torturadores': 1, 'bolsonaro': 1, 'bolsonaro..."


In [130]:
#carregando unlabeled data
df_u = pd.read_csv(r'C:/Users/camil/OneDrive/Área de Trabalho/mestrado/Unlabeled/bolsonaro.csv')
#df_u = df.sample(frac=1).reset_index(drop=True)
df_u

,Unnamed: 0,Text
0,0,isso me lembra eu anto e bruna discutindo c o ...
1,1,bolsonaro fdp acabando c a maconha esse país t...
2,2,Bolsonaro 2018? — No dia das eleições eu ainda...
3,3,Eu amo o Bolsonaro
4,4,acabou os seiscentao do bolsonaro entao amanha...
...,...,...
31990,31990,Eu conto 0 items dessa lista no governo Bolson...
31991,31991,Não existe comparação nenhuma entre os últimos...
31992,31992,Sendo sincero: seria injusto até comparar a ca...
31993,31993,Eu vou ser sinceros com vcs: eu não acho que n...


In [131]:
rows_train = len(df_train.index)
rows_test = len(df_test.index)
rows_label=rows_train+rows_test

In [132]:
rows_unlabel = len(df_u.index)
rows_unlabel

31995

In [133]:
frames = [df_train.Text, df_test.Text, df_u.Text]
X = pd.concat(frames)
X

0        Bolsonaro já tá avisando que vai acabar com a ...
1        HIPOCRISIA É MATO, HOMOFOBIA MATA, IGNORÂNCIA ...
2        Essa traíra pegou carona, está com inveja de J...
3        Não me deixem só!”: Fora de controle, Bolsonar...
4        ADEMAR EU FIZ UMAS POSTAGENS A RESPEITO HOJE D...
                               ...                        
31990    Eu conto 0 items dessa lista no governo Bolson...
31991    Não existe comparação nenhuma entre os últimos...
31992    Sendo sincero: seria injusto até comparar a ca...
31993    Eu vou ser sinceros com vcs: eu não acho que n...
31994    Esse vídeo da reunião dos ministros eh um absu...
Name: Text, Length: 41409, dtype: object

In [134]:
#vetoriza conjunto completo
from sklearn.feature_extraction.text import TfidfTransformer
#count_vect = CountVectorizer( )
#X_counts = count_vect.fit_transform(X)
#X_counts.shape

In [135]:
#tf_transformer = TfidfTransformer(use_idf=False).fit(X_counts)
from sklearn.feature_extraction.text import TfidfTransformer
vect = TfidfVectorizer(ngram_range=(2, 3),analyzer='char').fit(X)

X_tf = vect.transform(X)
X_tf.shape
X_ori = X_tf
#y_ori = df_train.Polarity
X_ori.shape

(41409, 26536)

In [136]:
x_train = X_ori[0:rows_train,:]
x_test = X_ori[rows_train:rows_train+rows_test,:]
x_test.shape


(2354, 26536)

In [137]:
#y_ori = df.Polarity
y_train = df_train.Polarity
y_test = df_test.Polarity
y_test.shape

(2354,)

In [138]:
#classificação
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
model = LogisticRegression(random_state=0)
model.fit(x_train,y_train)
pred_values = model.predict(x_test)     
acc = accuracy_score(pred_values , y_test)
acc

0.7880203908241291

In [139]:
f1_score(y_test, pred_values, average='macro')

0.7711577743598037

In [140]:
precision_score(y_test, pred_values, average='macro')

0.7931625569344525

In [141]:
recall_score(y_test, pred_values, average='macro')

0.7639249826749828

In [142]:
#predição dos não-rotulados
X_label = X_ori[rows_label:rows_label+ rows_unlabel]
pred_values_label= model.predict(X_label) 


In [143]:
#avalia a probabilidade de estar certo
prob_label = model.predict_proba(X_label) 
#np.savetxt('C:/Users/camil/OneDrive/Área de Trabalho/mestrado/Unlabeled/prob.csv', prob_label, delimiter=';')

In [144]:
#seleciona os itens que vão entrar no conjunto de treino
threshold = 0.24
t_size = int(threshold*rows_unlabel) #quanto vai adicionar a cada iteração
rows_train = x_train.shape[0]
rows_train

7060

In [145]:
reliable_pred_favor, reliable_pred_against  = [], []
for x_id, (prob, pred) in enumerate(zip(prob_label, pred_values_label)):
    #se 1, inclui no favor; senão, no against
    #incluir id, probabilidade e valor predito (1 ou 0)
    (reliable_pred_favor if (pred_values_label[x_id]) else reliable_pred_against).append((x_id, prob_label[x_id,pred_values_label[x_id]], pred_values_label[x_id]))
    

In [146]:
#coloca em ordem decrescente de confiabilidade a partir da probabilidade (coluna 2)
reliable_pred_favor = sorted(reliable_pred_favor, key=lambda x: x[1], reverse=True)
reliable_pred_against = sorted(reliable_pred_against, key=lambda x: x[1], reverse=True)

In [147]:
y= np.asarray(y_train)
index_list=[x for x in range(0, rows_train)] #indices para o novo X a partir do X original
iteractions = 0
position = 0
index_labels=[x for x in range(rows_label, rows_label+ rows_unlabel)]

while iteractions < 0.99:
    iteractions = iteractions + threshold
    for i in range(int(t_size/2)):
        #pega um a favor
        index = reliable_pred_favor[position][0] #descobre a posição no X_label 
        index_list.append(index+rows_label) #inclui na lista de indices
        y = np.append(y, 1) #empilha o y
        index_labels.remove(index+rows_label)
        #pega um contra
        index = reliable_pred_against[position][0] #descobre a posição no X_label 
        index_list.append(index+rows_label) #inclui na lista de indices
        y = np.append(y, 0) #empilha o y
        index_labels.remove(index+rows_label)
        #anda uma posição nos dois 
        position+=1

    X_train_novo = X_ori[index_list,:]#atualiza o X rotulado com os índices que estão em index_list
    X_label = X_ori[index_labels,:] #pega só os índices que sobraram
    rows_unlabel = X_label.get_shape()[0] #novo número de linhas sem rótulo
    rows_label = X_train_novo.get_shape()[0] + rows_test #linhas com rótulo novas + as de teste que estão guardadas
    X_ori = sparse.vstack([X_train_novo, x_test, X_label]) #empilha treino, teste e unlabelled
    index_labels=[x for x in range(rows_label, rows_label+ rows_unlabel)] #atualiza os unlabelled
    position = 0

   # x_train_novo, x_test_novo, y_train_novo, y_test_novo = train_test_split(
        #X_novo,
        #np.asarray(y),
        #test_size = 0.25,
        #random_state = 123,
       # shuffle = True,
        #stratify = np.asarray(y),
    #)

    model_novo = LogisticRegression(random_state=0)
    model_novo.fit(X_train_novo,y)
    #testa nos conjuntos originais
    pred_values_novo = model_novo.predict(x_test)     
    acc = accuracy_score(pred_values_novo , y_test)
    f1 = f1_score(y_test, pred_values_novo, average='macro')
    print("iteraction:",iteractions, "accuracy:", acc, "f1_score:", f1)
    
    pred_values_label= model_novo.predict(X_label) 
    prob_label = model.predict_proba(X_label) 
    reliable_pred_favor, reliable_pred_against  = [], []
    for x_id, (prob, pred) in enumerate(zip(prob_label, pred_values_label)):
        (reliable_pred_favor if (pred_values_label[x_id]) else reliable_pred_against).append((x_id, prob_label[x_id,pred_values_label[x_id]], pred_values_label[x_id]))
    
    reliable_pred_favor = sorted(reliable_pred_favor, key=lambda x: x[1], reverse=True)
    reliable_pred_against = sorted(reliable_pred_against, key=lambda x: x[1], reverse=True)    
    
    index_labels=[x for x in range(rows_label, rows_label+ rows_unlabel)]

iteraction: 0.24 accuracy: 0.7786745964316057 f1_score: 0.769847779845162
iteraction: 0.48 accuracy: 0.7595581988105352 f1_score: 0.7553147570270858


C:\Users\camil\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


iteraction: 0.72 accuracy: 0.7570093457943925 f1_score: 0.7535221116341528


IndexError: list index out of range